In [55]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
    .appName("Day20")\
    .getOrCreate()

In [58]:


!pip uninstall -y pyspark
!pip install pyspark==3.5.1

Found existing installation: pyspark 4.0.1
Uninstalling pyspark-4.0.1:
  Successfully uninstalled pyspark-4.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 8.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=a6adaa1d9661aa7c91bd2d97601d7601ea95f44cd1eacf25067c51b5a48c8212
  Stored in directory: /root/.cache/pip/wheels/b1/91/5f/283b53010a8016a4ff1c4a1edd99bbe73afacb099645b5471b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Uninstalling py4j-0.10.9.9:
      Successfully uninstalled py4j-0.10.9.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.1 requires pyspark[connect]~=4.0.0, b

In [ ]:
data=[
    ("U001","Delhi",450),
    ("U002","Mumbai",620),
    ("U003","Banglore",300),
    ("U004","Chennai",700)
]
columns=["user_id","city","amount"]
df=spark.createDataFrame(data,columns)
df.show()

+-------+--------+------+
|user_id|    city|amount|
+-------+--------+------+
|   U001|   Delhi|   450|
|   U002|  Mumbai|   620|
|   U003|Banglore|   300|
|   U004| Chennai|   700|
+-------+--------+------+



In [ ]:
from pyspark.sql.functions import col
df=df.withColumn("amount_with_tax",col("amount")*1.18)
df.show()

+-------+--------+------+-----------------+
|user_id|    city|amount|  amount_with_tax|
+-------+--------+------+-----------------+
|   U001|   Delhi|   450|            531.0|
|   U002|  Mumbai|   620|731.5999999999999|
|   U003|Banglore|   300|            354.0|
|   U004| Chennai|   700|            826.0|
+-------+--------+------+-----------------+



In [56]:
df=df.withColumnRenamed("amount_with_tax","total_amount")
df.show()

DateTimeException: [CANNOT_PARSE_TIMESTAMP] Text '2024/01/11' could not be parsed at index 4. Use `try_to_timestamp` to tolerate invalid input string and return NULL instead. SQLSTATE: 22007

In [ ]:
df=df.replace("Delhi","New Delhi",subset=["city"])
df.show()

+-------+---------+------+-----------------+
|user_id|     city|amount|     total_amount|
+-------+---------+------+-----------------+
|   U001|New Delhi|   450|            531.0|
|   U002|   Mumbai|   620|731.5999999999999|
|   U003| Banglore|   300|            354.0|
|   U004|  Chennai|   700|            826.0|
+-------+---------+------+-----------------+



In [ ]:
from pyspark.sql.functions import when
df=df.withColumn(
    "amount_category",
    when(col("amount")>=500,"High")
    .otherwise("Low")
)
df.show()

+-------+---------+------+-----------------+---------------+
|user_id|     city|amount|     total_amount|amount_category|
+-------+---------+------+-----------------+---------------+
|   U001|New Delhi|   450|            531.0|            Low|
|   U002|   Mumbai|   620|731.5999999999999|           High|
|   U003| Banglore|   300|            354.0|            Low|
|   U004|  Chennai|   700|            826.0|           High|
+-------+---------+------+-----------------+---------------+



In [ ]:
raw_orders = [
    ("ORD001","C001","Ravi"," Delhi ","Laptop","Electronics","45000","2024-01-05","Completed"),
    ("ORD002","C002","Sneha","Mumbai"," Mobile ","Electronics","32000","05/01/2024","Completed"),
    ("ORD003","C003","Aman","Bangalore","Laptop","Electronics","55000","2024/01/06","Completed"),
    ("ORD004","C004","Pooja","Delhi","Tablet"," Electronics ","","2024-01-07","Cancelled"),
    ("ORD005","C005","Neha","Chennai","Laptop","Electronics","48000","invalid_date","Completed"),
    ("ORD006","C006","Rahul","Mumbai","Mobile","Electronics",None,"2024-01-08","Completed"),
    ("ORD007","C007","Kiran","Bangalore","Tablet","Electronics","30000","2024-01-08","Completed"),
    ("ORD008","C008","Amit","Delhi","Laptop","electronics","45000","2024-01-09","Completed"),
    ("ORD009","C009","Priya"," Pune","Mobile","Electronics","28000","09-01-2024","Completed"),
    ("ORD010","C010","Suresh","Mumbai","Laptop","Electronics","55000","2024-01-10","Completed"),
    ("ORD010","C010","Suresh","Mumbai","Laptop","Electronics","55000","2024-01-10","Completed"),  # duplicate
    ("ORD011","C011","Meena","Chennai","Tablet","Electronics","31000","2024-01-11","Completed"),
    ("ORD012","C012","Arjun","Delhi","Mobile","Electronics","27000","2024/01/11","Completed"),
    ("ORD013","C013","Nikhil","Bangalore","Laptop","Electronics","60000","2024-01-12","Completed"),
    ("ORD014","C014","Rohit","Mumbai","Mobile","Electronics","invalid_price","2024-01-12","Completed"),
    ("ORD015","C015","Anita","Delhi","Tablet","Electronics","29000","2024-01-13","Completed"),
    ("ORD016","C016","Vikas","Chennai","Laptop","Electronics","52000","2024-01-13","Completed"),
    ("ORD017","C017","Sunita","Mumbai","Mobile","Electronics","33000","2024-01-14","Completed"),
    ("ORD018","C018","Deepak","Bangalore","Laptop","Electronics","58000","2024-01-14","Completed"),
    ("ORD019","C019","Pallavi","Delhi","Mobile","Electronics","26000","2024-01-15","Completed"),
    ("ORD020","C020","Manish","Mumbai","Tablet","Electronics","34000","2024-01-15","Completed")
]
columns=["order id","customer id","customer name","city","product","category","price","order_date","status"]
df=spark.createDataFrame(raw_orders,columns)
df.show()


+--------+-----------+-------------+---------+--------+-------------+-------------+------------+---------+
|order id|customer id|customer name|     city| product|     category|        price|  order_date|   status|
+--------+-----------+-------------+---------+--------+-------------+-------------+------------+---------+
|  ORD001|       C001|         Ravi|   Delhi |  Laptop|  Electronics|        45000|  2024-01-05|Completed|
|  ORD002|       C002|        Sneha|   Mumbai| Mobile |  Electronics|        32000|  05/01/2024|Completed|
|  ORD003|       C003|         Aman|Bangalore|  Laptop|  Electronics|        55000|  2024/01/06|Completed|
|  ORD004|       C004|        Pooja|    Delhi|  Tablet| Electronics |             |  2024-01-07|Cancelled|
|  ORD005|       C005|         Neha|  Chennai|  Laptop|  Electronics|        48000|invalid_date|Completed|
|  ORD006|       C006|        Rahul|   Mumbai|  Mobile|  Electronics|         NULL|  2024-01-08|Completed|
|  ORD007|       C007|        Kiran|B

In [ ]:
df=df.withColumnRenamed("order id","order_id")\
    .withColumnRenamed("customer id","customer_id")\
    .withColumnRenamed("customer name","customer_name")\
    .withColumnRenamed("order date","order_date")
df.show()

+--------+-----------+-------------+---------+--------+-------------+-------------+------------+---------+
|order_id|customer_id|customer_name|     city| product|     category|        price|  order_date|   status|
+--------+-----------+-------------+---------+--------+-------------+-------------+------------+---------+
|  ORD001|       C001|         Ravi|   Delhi |  Laptop|  Electronics|        45000|  2024-01-05|Completed|
|  ORD002|       C002|        Sneha|   Mumbai| Mobile |  Electronics|        32000|  05/01/2024|Completed|
|  ORD003|       C003|         Aman|Bangalore|  Laptop|  Electronics|        55000|  2024/01/06|Completed|
|  ORD004|       C004|        Pooja|    Delhi|  Tablet| Electronics |             |  2024-01-07|Cancelled|
|  ORD005|       C005|         Neha|  Chennai|  Laptop|  Electronics|        48000|invalid_date|Completed|
|  ORD006|       C006|        Rahul|   Mumbai|  Mobile|  Electronics|         NULL|  2024-01-08|Completed|
|  ORD007|       C007|        Kiran|B

In [ ]:
from pyspark.sql import functions as F
# Add price_with_tax (18%)
df = df.withColumn("price_with_tax",F.expr("try_cast(price as double)*1.18"))
df.show()

+--------+-----------+-------------+---------+--------+-------------+-------------+------------+---------+--------------+
|order_id|customer_id|customer_name|     city| product|     category|        price|  order_date|   status|price_with_tax|
+--------+-----------+-------------+---------+--------+-------------+-------------+------------+---------+--------------+
|  ORD001|       C001|         Ravi|   Delhi |  Laptop|  Electronics|        45000|  2024-01-05|Completed|       53100.0|
|  ORD002|       C002|        Sneha|   Mumbai| Mobile |  Electronics|        32000|  05/01/2024|Completed|       37760.0|
|  ORD003|       C003|         Aman|Bangalore|  Laptop|  Electronics|        55000|  2024/01/06|Completed|       64900.0|
|  ORD004|       C004|        Pooja|    Delhi|  Tablet| Electronics |             |  2024-01-07|Cancelled|          NULL|
|  ORD005|       C005|         Neha|  Chennai|  Laptop|  Electronics|        48000|invalid_date|Completed|       56640.0|
|  ORD006|       C006|  

In [ ]:
from pyspark.sql import functions as F
df=df.withColumn("price_category",
           F.when(F.expr("try_cast(price as double)")<500,"Low")
          .when((F.expr("try_cast(price as double)")>=500)&(F.expr("try_cast(price as double)")<1500),"Medium")
          .otherwise("High"))
df.show()

+--------+-----------+-------------+---------+--------+-------------+-------------+------------+---------+--------------+--------------+
|order_id|customer_id|customer_name|     city| product|     category|        price|  order_date|   status|price_with_tax|price_category|
+--------+-----------+-------------+---------+--------+-------------+-------------+------------+---------+--------------+--------------+
|  ORD001|       C001|         Ravi|   Delhi |  Laptop|  Electronics|        45000|  2024-01-05|Completed|       53100.0|          High|
|  ORD002|       C002|        Sneha|   Mumbai| Mobile |  Electronics|        32000|  05/01/2024|Completed|       37760.0|          High|
|  ORD003|       C003|         Aman|Bangalore|  Laptop|  Electronics|        55000|  2024/01/06|Completed|       64900.0|          High|
|  ORD004|       C004|        Pooja|    Delhi|  Tablet| Electronics |             |  2024-01-07|Cancelled|          NULL|          High|
|  ORD005|       C005|         Neha|  Che

In [ ]:
from pyspark.sql import functions as F
df=df.withColumn("city",F.initcap(F.trim(F.col("city"))))\
     .withColumn("product",F.initcap(F.trim(F.col("product"))))\
     .withColumn("category",F.initcap(F.trim(F.col("category"))))
df.show()


+--------+-----------+-------------+---------+-------+-----------+-------------+------------+---------+--------------+--------------+
|order_id|customer_id|customer_name|     city|product|   category|        price|  order_date|   status|price_with_tax|price_category|
+--------+-----------+-------------+---------+-------+-----------+-------------+------------+---------+--------------+--------------+
|  ORD001|       C001|         Ravi|    Delhi| Laptop|Electronics|        45000|  2024-01-05|Completed|       53100.0|          High|
|  ORD002|       C002|        Sneha|   Mumbai| Mobile|Electronics|        32000|  05/01/2024|Completed|       37760.0|          High|
|  ORD003|       C003|         Aman|Bangalore| Laptop|Electronics|        55000|  2024/01/06|Completed|       64900.0|          High|
|  ORD004|       C004|        Pooja|    Delhi| Tablet|Electronics|             |  2024-01-07|Cancelled|          NULL|          High|
|  ORD005|       C005|         Neha|  Chennai| Laptop|Electron

In [ ]:
df=df.withColumn("price_num",F.expr("try_cast(regexp_replace(price,'[^0-9]','') as int)"))
df.show()

+--------+-----------+-------------+---------+-------+-----------+-------------+------------+---------+--------------+--------------+---------+
|order_id|customer_id|customer_name|     city|product|   category|        price|  order_date|   status|price_with_tax|price_category|price_num|
+--------+-----------+-------------+---------+-------+-----------+-------------+------------+---------+--------------+--------------+---------+
|  ORD001|       C001|         Ravi|    Delhi| Laptop|Electronics|        45000|  2024-01-05|Completed|       53100.0|          High|    45000|
|  ORD002|       C002|        Sneha|   Mumbai| Mobile|Electronics|        32000|  05/01/2024|Completed|       37760.0|          High|    32000|
|  ORD003|       C003|         Aman|Bangalore| Laptop|Electronics|        55000|  2024/01/06|Completed|       64900.0|          High|    55000|
|  ORD004|       C004|        Pooja|    Delhi| Tablet|Electronics|             |  2024-01-07|Cancelled|          NULL|          High|   

In [ ]:
df=df.fillna({"price_num":0})
df.show()

+--------+-----------+-------------+---------+-------+-----------+-------------+------------+---------+--------------+--------------+---------+
|order_id|customer_id|customer_name|     city|product|   category|        price|  order_date|   status|price_with_tax|price_category|price_num|
+--------+-----------+-------------+---------+-------+-----------+-------------+------------+---------+--------------+--------------+---------+
|  ORD001|       C001|         Ravi|    Delhi| Laptop|Electronics|        45000|  2024-01-05|Completed|       53100.0|          High|    45000|
|  ORD002|       C002|        Sneha|   Mumbai| Mobile|Electronics|        32000|  05/01/2024|Completed|       37760.0|          High|    32000|
|  ORD003|       C003|         Aman|Bangalore| Laptop|Electronics|        55000|  2024/01/06|Completed|       64900.0|          High|    55000|
|  ORD004|       C004|        Pooja|    Delhi| Tablet|Electronics|             |  2024-01-07|Cancelled|          NULL|          High|   

In [ ]:
from pyspark.sql import functions as F

clean = F.regexp_replace(F.trim(F.col("order_date")), r"[./]", "-")

df = df.withColumn(
    "order_dt",
    F.when(clean.rlike(r"^\d{4}-\d{2}-\d{2}$"), F.to_date(clean, "yyyy-MM-dd"))
     .when(clean.rlike(r"^\d{2}-\d{2}-\d{4}$"), F.to_date(clean, "dd-MM-yyyy"))
     .otherwise(F.lit(None).cast("date"))
)

df.select("order_date", "order_dt").show(truncate=False)

DateTimeException: [CANNOT_PARSE_TIMESTAMP] Text '2024/01/11' could not be parsed at index 4. Use `try_to_timestamp` to tolerate invalid input string and return NULL instead. SQLSTATE: 22007

In [ ]:
df=df.dropDuplicates(["order_id"])
df.show()

DateTimeException: [CANNOT_PARSE_TIMESTAMP] Text '2024/01/11' could not be parsed at index 4. Use `try_to_timestamp` to tolerate invalid input string and return NULL instead. SQLSTATE: 22007

In [ ]:
df = df.filter(F.col("order_status") == "Completed")
df.show()

{"ts": "2025-12-22 06:58:03.925", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `order_status` cannot be resolved. Did you mean one of the following? [`order_date`, `order_dt`, `order_id`, `status`, `product`]. SQLSTATE: 42703", "context": {"file": "line 1 in cell [53]", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o2097.filter.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `order_status` cannot be resolved. Did you mean one of the following? [`order_date`, `order_dt`, `order_id`, `status`, `product`]. SQLSTATE: 42703;\n'Filter '`=`('order_status, Completed)\n+- Deduplicate [order_id#147]\n   +- Project [order_id#147, customer_id#148, customer_name#149, city#415, product#416, cat

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `order_status` cannot be resolved. Did you mean one of the following? [`order_date`, `order_dt`, `order_id`, `status`, `product`]. SQLSTATE: 42703;
'Filter '`=`('order_status, Completed)
+- Deduplicate [order_id#147]
   +- Project [order_id#147, customer_id#148, customer_name#149, city#415, product#416, category#417, price#116, order_date#713, status#118, price_with_tax#243, price_category#380, price_num#490, CASE WHEN RLIKE(regexp_replace(trim(cast(order_date#713 as string), None), [./], -, 1), ^\d{4}-\d{2}-\d{2}$) THEN to_date(regexp_replace(trim(cast(order_date#713 as string), None), [./], -, 1), Some(yyyy-MM-dd), Some(Etc/UTC), true) WHEN RLIKE(regexp_replace(trim(cast(order_date#713 as string), None), [./], -, 1), ^\d{2}-\d{2}-\d{4}$) THEN to_date(regexp_replace(trim(cast(order_date#713 as string), None), [./], -, 1), Some(dd-MM-yyyy), Some(Etc/UTC), true) ELSE cast(null as d